## Sample Model

Based on the exploration, we try the following *as a starting point*.

> #### Delete the following columns
> - days_with_fog
> - direction_peak_wind_speed
> - direction_max_wind_speed
> - max_wind_speed
> - year_built
> - days_above_90F
> - days_above_110F
> - facility_type
> - site_eui (obviously, because target column)
> - For now also delete Year_Factor

> #### Impute missing values for energy_star_rating 
> Imputing is done by replacing nan by the mean <br>
> By second thoughts we do NOT impute as XGBoost infers missing values

> #### One-hot encode categorical values 
> - State_Factor
> - Building_Class

For the sample model, we use **random forest** and **xgboost**.

In [1]:
import sklearn
import xgboost as xgb
from evaluation import RMSE
from evaluation import helper_func
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error

#### Preparing and reading data

In [2]:
data = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
submission = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/test.csv")

In [3]:
len(submission)

9705

### Feature engineering

In [4]:
len(data.facility_type.unique())

60

#### delete columns

In [5]:
reduced = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")

to_delete = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed", "year_built", 
             "days_above_90F", "days_above_110F", "facility_type", "Year_Factor"]

# to_delete_less = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed"]
to_delete_less = ["days_with_fog", "direction_peak_wind_speed", "max_wind_speed"]

def delete_cols(dataframe, columns):
    for colname in columns:
        del dataframe[colname]

delete_cols(reduced, to_delete_less)
delete_cols(submission, to_delete_less)

# # collect garbage
# gc.collect()


#### Impute variables 

In [6]:
# impute with mean, ensures highest correlation

reduced['energy_star_rating'] = reduced['energy_star_rating'].fillna(56.0)
submission['energy_star_rating'] = submission['energy_star_rating'].fillna(56.0)
reduced['year_built'] = reduced['year_built'].fillna(reduced['year_built'].median())
submission['year_built'] = submission['year_built'].fillna(submission['year_built'].median())

In [7]:
fillmedian = ["direction_max_wind_speed"]

for i in fillmedian:
    reduced[i] = reduced[i].fillna(reduced[i].median())
    submission[i] = submission[i].fillna(submission[i].median())

In [8]:
# test which value enhances highest correlation still
# highest_corr = []

# for i in data['energy_star_rating'].unique():
#     d = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
#     d['energy_star_rating'] = d['energy_star_rating'].fillna(i)
#     correlations = helper_func.get_correlation(d, 'site_eui', 0)[1]
#     highest_corr.append((i, correlations[1]))

#### Create new columns

In [9]:
# below compute sum of below 30 and sum of above 80
days_ = [string for string in reduced.columns if 'days_' in string]

In [10]:
# # below compute sum of below 30 and sum of above 80
# reduced['d_below_30_F'] = reduced[list(reduced.filter(regex='days_below_'))].sum(axis=1)
# reduced['d_above_80_F'] = reduced[list(reduced.filter(regex='days_above_'))].sum(axis=1)
# submission['d_below_30_F'] = submission[list(submission.filter(regex='days_below_'))].sum(axis=1)
# submission['d_above_80_F'] = submission[list(submission.filter(regex='days_above_'))].sum(axis=1)

In [11]:
# delete the other columns
# delete_cols(reduced, days_)
# delete_cols(submission, days_)

#### Onehot encoding

In [12]:
onehot = ["facility_type", "State_Factor", "building_class"]

def onehotter(dataframe, to_onehot):
    
    for i in to_onehot:
        ohe_df = pd.get_dummies(dataframe[i], prefix=i)

        # concat with original data
        dataframe = pd.concat([dataframe, ohe_df], axis=1).drop([i], axis=1)
        
    return dataframe

In [13]:
submission = onehotter(submission, onehot)
reduced = onehotter(reduced, onehot)

In [14]:
# save and pop submission id apart
Y_test_sub = submission["id"]
submission.pop("id")

0       75757
1       75758
2       75759
3       75760
4       75761
        ...  
9700    85457
9701    85458
9702    85459
9703    85460
9704    85461
Name: id, Length: 9705, dtype: int64

## Sample model

In [15]:
path = "/Users/charlottefelius/documents/WIDS2022/WIDS/data/train.csv"

#### Split into train and test set about  30 / 70 ratio

In [16]:
X = reduced
y = X['site_eui']
y_id = X['id']
X.pop('site_eui')
X.pop('id')
X.pop('State_Factor_State_6')

0        0
1        0
2        0
3        0
4        0
        ..
75752    0
75753    0
75754    0
75755    0
75756    0
Name: State_Factor_State_6, Length: 75757, dtype: uint8

In [17]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=123)

In [19]:
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 100, alpha = 0.2, n_estimators = 40)

In [20]:
# Highest score RMSE: 38.672019

model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 38.789511


In [21]:
# array([ 84.76915,  68.39229, 164.82307, ..., 136.7066 ,  65.37906,
#        136.15688], dtype=float32)

preds

array([ 76.47513 ,  73.636444,  60.30724 , ...,  53.62061 ,  66.924416,
       124.64494 ], dtype=float32)

In [22]:
raise NotImplementedError()

NotImplementedError: 

In [ ]:
# Now fit on whole training set
model.fit(X, y)
preds = model.predict(submission)

In [ ]:
# Convert Preds to DataFrame 

result = pd.DataFrame(Y_test_sub)
result["site_eui"] = preds

In [ ]:
# Write to CSV file

result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/first_attempt9.csv', index=False, header=True)